# Kaggle Contest ----> House Price Predictions 

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/description

We are given many features that a person wants to have in his house. On the basis of these feature, price of house is estimated.

So, here we are given a number of features(almost 80) and we need to predict the price of the house.

## 1) Importing the libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.functional as F

## 2) Checking for GPU

In [2]:
torch.cuda.is_available()

True

## 3) Data Preprocessing

For train file

In [3]:
df = pd.read_csv('drive/My Drive/Pytorch_DataSet/house-prices/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
len(df)

1460

For Test file

In [5]:
# For test file

df_test = pd.read_csv('drive/My Drive/Pytorch_DataSet/house-prices/test.csv')
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
len(df_test)

1459

### 3.1) Handling missing values

For train file

In [7]:
# Checking for null values

cols = np.array(df.columns)
for col in cols:
  print(col , df[col].isnull().sum())

Id 0
MSSubClass 0
MSZoning 0
LotFrontage 259
LotArea 0
Street 0
Alley 1369
LotShape 0
LandContour 0
Utilities 0
LotConfig 0
LandSlope 0
Neighborhood 0
Condition1 0
Condition2 0
BldgType 0
HouseStyle 0
OverallQual 0
OverallCond 0
YearBuilt 0
YearRemodAdd 0
RoofStyle 0
RoofMatl 0
Exterior1st 0
Exterior2nd 0
MasVnrType 8
MasVnrArea 8
ExterQual 0
ExterCond 0
Foundation 0
BsmtQual 37
BsmtCond 37
BsmtExposure 38
BsmtFinType1 37
BsmtFinSF1 0
BsmtFinType2 38
BsmtFinSF2 0
BsmtUnfSF 0
TotalBsmtSF 0
Heating 0
HeatingQC 0
CentralAir 0
Electrical 1
1stFlrSF 0
2ndFlrSF 0
LowQualFinSF 0
GrLivArea 0
BsmtFullBath 0
BsmtHalfBath 0
FullBath 0
HalfBath 0
BedroomAbvGr 0
KitchenAbvGr 0
KitchenQual 0
TotRmsAbvGrd 0
Functional 0
Fireplaces 0
FireplaceQu 690
GarageType 81
GarageYrBlt 81
GarageFinish 81
GarageCars 0
GarageArea 0
GarageQual 81
GarageCond 81
PavedDrive 0
WoodDeckSF 0
OpenPorchSF 0
EnclosedPorch 0
3SsnPorch 0
ScreenPorch 0
PoolArea 0
PoolQC 1453
Fence 1179
MiscFeature 1406
MiscVal 0
MoSold 0
YrSol

In [8]:
# Removing the columns that contains entries above 500 as Nan.

cols_to_del = []
for col in cols:
  if df[col].isnull().sum() > 500:
    cols_to_del.append(col)

cols_to_del

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [0]:
for col in cols_to_del:
  df.drop(col, axis = 1, inplace = True)

In [0]:
cols = np.array(df.columns)
cols_to_fill_missing_values = []
for col in cols:
  #print(col,  df[col].isnull().sum())
  if df[col].isnull().sum() > 0:
    cols_to_fill_missing_values.append((col, df[col].dtype))

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [12]:
cols_to_fill_missing_values

[('LotFrontage', dtype('float64')),
 ('MasVnrType', dtype('O')),
 ('MasVnrArea', dtype('float64')),
 ('BsmtQual', dtype('O')),
 ('BsmtCond', dtype('O')),
 ('BsmtExposure', dtype('O')),
 ('BsmtFinType1', dtype('O')),
 ('BsmtFinType2', dtype('O')),
 ('Electrical', dtype('O')),
 ('GarageType', dtype('O')),
 ('GarageYrBlt', dtype('float64')),
 ('GarageFinish', dtype('O')),
 ('GarageQual', dtype('O')),
 ('GarageCond', dtype('O'))]

- Now, fill all the columns other than Object Datatype by the mean of the remaining values.
- Deleting all the rows that have null values of object datatype.

In [0]:
df['LotFrontage'].fillna(df['LotFrontage'].mean(),inplace = True)
df['MasVnrArea'].fillna(df['MasVnrArea'].mean(),inplace = True)
df['GarageYrBlt'].fillna(df['GarageYrBlt'].mean(),inplace = True)
df.dropna(inplace=True)


In [14]:
"""
cols = np.array(df.columns)
for col in cols:
  print(col, df[col].isnull().sum())

"""  

'\ncols = np.array(df.columns)\nfor col in cols:\n  print(col, df[col].isnull().sum())\n\n'

For Test file

In [0]:
# Checking for null values

cols_test = np.array(df_test.columns)

# Removing the columns that contains entries above 500 as Nan.

cols_to_del_test = []
for col in cols_test:
  if df_test[col].isnull().sum() > 500:
    cols_to_del_test.append(col)

for col in cols_to_del_test:
  df_test.drop(col, axis = 1, inplace = True)

cols_test = np.array(df_test.columns)
cols_to_fill_missing_values_test = []
for col in cols_test:
  #print(col,  df[col].isnull().sum())
  if df_test[col].isnull().sum() > 0:
    cols_to_fill_missing_values_test.append((col, df_test[col].dtype))

df_test['LotFrontage'].fillna(df_test['LotFrontage'].mean(),inplace = True)
df_test['MasVnrArea'].fillna(df_test['MasVnrArea'].mean(),inplace = True)
df_test['GarageYrBlt'].fillna(df_test['GarageYrBlt'].mean(),inplace = True)
df_test.dropna(inplace=True)


In [16]:
for col in cols_test:
  print(col,  df[col].isnull().sum())

Id 0
MSSubClass 0
MSZoning 0
LotFrontage 0
LotArea 0
Street 0
LotShape 0
LandContour 0
Utilities 0
LotConfig 0
LandSlope 0
Neighborhood 0
Condition1 0
Condition2 0
BldgType 0
HouseStyle 0
OverallQual 0
OverallCond 0
YearBuilt 0
YearRemodAdd 0
RoofStyle 0
RoofMatl 0
Exterior1st 0
Exterior2nd 0
MasVnrType 0
MasVnrArea 0
ExterQual 0
ExterCond 0
Foundation 0
BsmtQual 0
BsmtCond 0
BsmtExposure 0
BsmtFinType1 0
BsmtFinSF1 0
BsmtFinType2 0
BsmtFinSF2 0
BsmtUnfSF 0
TotalBsmtSF 0
Heating 0
HeatingQC 0
CentralAir 0
Electrical 0
1stFlrSF 0
2ndFlrSF 0
LowQualFinSF 0
GrLivArea 0
BsmtFullBath 0
BsmtHalfBath 0
FullBath 0
HalfBath 0
BedroomAbvGr 0
KitchenAbvGr 0
KitchenQual 0
TotRmsAbvGrd 0
Functional 0
Fireplaces 0
GarageType 0
GarageYrBlt 0
GarageFinish 0
GarageCars 0
GarageArea 0
GarageQual 0
GarageCond 0
PavedDrive 0
WoodDeckSF 0
OpenPorchSF 0
EnclosedPorch 0
3SsnPorch 0
ScreenPorch 0
PoolArea 0
MiscVal 0
MoSold 0
YrSold 0
SaleType 0
SaleCondition 0


### 3.2) Handling of categorical, continuous and label values



In [17]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [18]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPo

In [19]:
# For train file

cols = np.array(df.columns)
print(len(cols))
cols = np.delete(cols, [0,75])
print(cols)

# For test file

cols_test = np.array(df_test.columns)
cols_test = np.delete(cols_test, [0,75])

76
['MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'LotShape'
 'LandContour' 'Utilities' 'LotConfig' 'LandSlope' 'Neighborhood'
 'Condition1' 'Condition2' 'BldgType' 'HouseStyle' 'OverallQual'
 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'RoofStyle' 'RoofMatl'
 'Exterior1st' 'Exterior2nd' 'MasVnrType' 'MasVnrArea' 'ExterQual'
 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' '1stFlrSF'
 '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath'
 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual'
 'TotRmsAbvGrd' 'Functional' 'Fireplaces' 'GarageType' 'GarageYrBlt'
 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageQual' 'GarageCond'
 'PavedDrive' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch' '3SsnPorch'
 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold' 'YrSold' 'SaleType'
 'SaleCondition']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  # Remove the CWD from sys.path while we load stuff.


In [20]:
# For train file

y_cols = ['SalePrice']
categorical_cols = []
continous_cols = []

for col in cols:
  if df[col].dtype == 'O':
    categorical_cols.append(col)
  else:
    continous_cols.append(col)

print(len(categorical_cols), categorical_cols)
print(len(continous_cols), continous_cols)
print(len(y_cols), y_cols) 

# For test file

y_cols_test = ['SalePrice']
categorical_cols_test = []
continous_cols_test = []

for col in cols_test:
  if df_test[col].dtype == 'O':
    categorical_cols_test.append(col)
  else:
    continous_cols_test.append(col)

print(len(categorical_cols_test), categorical_cols_test)
print(len(continous_cols_test), continous_cols_test)
print(len(y_cols_test), y_cols_test) 

38 ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
36 ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'M

In [0]:
# Changing the datatype of categorical data to `category`

# For train file

for col in categorical_cols:
  df[col] = df[col].astype('category')  

# For test file

for col in categorical_cols_test:
  df_test[col] = df_test[col].astype('category')  


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1338 non-null   int64   
 1   MSSubClass     1338 non-null   int64   
 2   MSZoning       1338 non-null   category
 3   LotFrontage    1338 non-null   float64 
 4   LotArea        1338 non-null   int64   
 5   Street         1338 non-null   category
 6   LotShape       1338 non-null   category
 7   LandContour    1338 non-null   category
 8   Utilities      1338 non-null   category
 9   LotConfig      1338 non-null   category
 10  LandSlope      1338 non-null   category
 11  Neighborhood   1338 non-null   category
 12  Condition1     1338 non-null   category
 13  Condition2     1338 non-null   category
 14  BldgType       1338 non-null   category
 15  HouseStyle     1338 non-null   category
 16  OverallQual    1338 non-null   int64   
 17  OverallCond    1338 non-null   in

For Categorical Data

In [23]:
# For train data

cats = []
for col in categorical_cols:
  cats.append(df[col].cat.codes.values)
cats = np.array(cats)  
cats = cats.transpose()
print(cats.shape, cats)

# For test data

cats_test = []
for col in categorical_cols_test:
  cats_test.append(df_test[col].cat.codes.values)
cats_test = np.array(cats_test)  
cats_test = cats_test.transpose()
print(cats_test.shape, cats_test)

(1338, 38) [[3 1 3 ... 2 8 4]
 [3 1 3 ... 2 8 4]
 [3 1 0 ... 2 8 4]
 ...
 [3 1 3 ... 2 8 4]
 [3 1 3 ... 2 8 4]
 [3 1 3 ... 2 8 4]]
(1319, 38) [[2 1 3 ... 2 8 4]
 [3 1 0 ... 2 8 4]
 [3 1 0 ... 2 8 4]
 ...
 [4 1 3 ... 2 8 0]
 [3 1 3 ... 2 8 0]
 [3 1 3 ... 2 8 4]]


For Continous Data

In [24]:
# For train data

conts = []
for col in continous_cols:
  conts.append(df[col].values)

conts = np.array(conts)
conts = conts.transpose()
print(conts.shape, conts)

# For test data

conts_test = []
for col in continous_cols_test:
  conts_test.append(df_test[col].values)

conts_test = np.array(conts_test)
conts_test = conts_test.transpose()
print(conts_test.shape, conts_test)

(1338, 36) [[6.000e+01 6.500e+01 8.450e+03 ... 0.000e+00 2.000e+00 2.008e+03]
 [2.000e+01 8.000e+01 9.600e+03 ... 0.000e+00 5.000e+00 2.007e+03]
 [6.000e+01 6.800e+01 1.125e+04 ... 0.000e+00 9.000e+00 2.008e+03]
 ...
 [7.000e+01 6.600e+01 9.042e+03 ... 2.500e+03 5.000e+00 2.010e+03]
 [2.000e+01 6.800e+01 9.717e+03 ... 0.000e+00 4.000e+00 2.010e+03]
 [2.000e+01 7.500e+01 9.937e+03 ... 0.000e+00 6.000e+00 2.008e+03]]
(1319, 36) [[2.0000e+01 8.0000e+01 1.1622e+04 ... 0.0000e+00 6.0000e+00 2.0100e+03]
 [2.0000e+01 8.1000e+01 1.4267e+04 ... 1.2500e+04 6.0000e+00 2.0100e+03]
 [6.0000e+01 7.4000e+01 1.3830e+04 ... 0.0000e+00 3.0000e+00 2.0100e+03]
 ...
 [1.6000e+02 2.1000e+01 1.8940e+03 ... 0.0000e+00 4.0000e+00 2.0060e+03]
 [2.0000e+01 1.6000e+02 2.0000e+04 ... 0.0000e+00 9.0000e+00 2.0060e+03]
 [6.0000e+01 7.4000e+01 9.6270e+03 ... 0.0000e+00 1.1000e+01 2.0060e+03]]


Converting the categorical and continous data to one-hot encoder

In [25]:
# For train data

categorical = np.stack(cats, axis=0)
print(categorical)

# For test data

categorical_test = np.stack(cats_test, axis=0)
print(categorical_test)


[[3 1 3 ... 2 8 4]
 [3 1 3 ... 2 8 4]
 [3 1 0 ... 2 8 4]
 ...
 [3 1 3 ... 2 8 4]
 [3 1 3 ... 2 8 4]
 [3 1 3 ... 2 8 4]]
[[2 1 3 ... 2 8 4]
 [3 1 0 ... 2 8 4]
 [3 1 0 ... 2 8 4]
 ...
 [4 1 3 ... 2 8 0]
 [3 1 3 ... 2 8 0]
 [3 1 3 ... 2 8 4]]


In [26]:
# For train data

continous = np.stack(conts, axis = 0)
print(continous)

# For test data

continous_test = np.stack(conts_test, axis = 0)
print(continous_test)

[[6.000e+01 6.500e+01 8.450e+03 ... 0.000e+00 2.000e+00 2.008e+03]
 [2.000e+01 8.000e+01 9.600e+03 ... 0.000e+00 5.000e+00 2.007e+03]
 [6.000e+01 6.800e+01 1.125e+04 ... 0.000e+00 9.000e+00 2.008e+03]
 ...
 [7.000e+01 6.600e+01 9.042e+03 ... 2.500e+03 5.000e+00 2.010e+03]
 [2.000e+01 6.800e+01 9.717e+03 ... 0.000e+00 4.000e+00 2.010e+03]
 [2.000e+01 7.500e+01 9.937e+03 ... 0.000e+00 6.000e+00 2.008e+03]]
[[2.0000e+01 8.0000e+01 1.1622e+04 ... 0.0000e+00 6.0000e+00 2.0100e+03]
 [2.0000e+01 8.1000e+01 1.4267e+04 ... 1.2500e+04 6.0000e+00 2.0100e+03]
 [6.0000e+01 7.4000e+01 1.3830e+04 ... 0.0000e+00 3.0000e+00 2.0100e+03]
 ...
 [1.6000e+02 2.1000e+01 1.8940e+03 ... 0.0000e+00 4.0000e+00 2.0060e+03]
 [2.0000e+01 1.6000e+02 2.0000e+04 ... 0.0000e+00 9.0000e+00 2.0060e+03]
 [6.0000e+01 7.4000e+01 9.6270e+03 ... 0.0000e+00 1.1000e+01 2.0060e+03]]


Converting everything to tensors.

In [28]:
# For train data

categorical = torch.tensor(categorical, dtype = torch.int64)
continous = torch.tensor(continous, dtype = torch.float)
y = torch.tensor(np.log(df[y_cols].values), dtype= torch.float).reshape(-1,1)
print(len(categorical), len(continous), len(y), categorical.shape, continous.shape, y.shape)

# For test data

categorical_test = torch.tensor(categorical_test, dtype = torch.int64)
continous_test = torch.tensor(continous_test, dtype = torch.float)
print(len(categorical_test), len(continous_test), categorical_test.shape, continous_test.shape)


1338 1338 1338 torch.Size([1338, 38]) torch.Size([1338, 36]) torch.Size([1338, 1])
1319 1319 torch.Size([1319, 38]) torch.Size([1319, 36])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a ten

### 3.3) Working on Embeddings

In [29]:
# Taking the size of each category

# For train data

cats = [len(df[col].cat.categories) for col in categorical_cols]
print(np.array(cats).flatten())

# For test data

cats_test = [len(df_test[col].cat.categories) for col in categorical_cols_test]
print(np.array(cats_test).flatten())

[ 5  2  4  4  2  5  3 25  9  8  5  8  6  8 14 16  4  4  4  5  4  4  4  6
  6  4  5  2  5  4  7  6  3  5  5  3  9  6]
[ 5  2  4  4  1  5  3 25  9  5  5  7  6  4 11 12  4  4  5  5  4  4  4  6
  6  2  5  2  4  4  6  6  3  4  5  3  9  6]


In [30]:
# For train data

emb_size = [(size,min((size+1)//2, 50)) for size in cats]
print(emb_size)

# For test data

emb_size_test = [(size,min((size+1)//2, 50)) for size in cats_test]
print(emb_size_test)

[(5, 3), (2, 1), (4, 2), (4, 2), (2, 1), (5, 3), (3, 2), (25, 13), (9, 5), (8, 4), (5, 3), (8, 4), (6, 3), (8, 4), (14, 7), (16, 8), (4, 2), (4, 2), (4, 2), (5, 3), (4, 2), (4, 2), (4, 2), (6, 3), (6, 3), (4, 2), (5, 3), (2, 1), (5, 3), (4, 2), (7, 4), (6, 3), (3, 2), (5, 3), (5, 3), (3, 2), (9, 5), (6, 3)]
[(5, 3), (2, 1), (4, 2), (4, 2), (1, 1), (5, 3), (3, 2), (25, 13), (9, 5), (5, 3), (5, 3), (7, 4), (6, 3), (4, 2), (11, 6), (12, 6), (4, 2), (4, 2), (5, 3), (5, 3), (4, 2), (4, 2), (4, 2), (6, 3), (6, 3), (2, 1), (5, 3), (2, 1), (4, 2), (4, 2), (6, 3), (6, 3), (3, 2), (4, 2), (5, 3), (3, 2), (9, 5), (6, 3)]


In [0]:
selfembeds = nn.ModuleList([nn.Embedding(n1,n2) for n1,n2 in emb_size])

In [32]:
embeddingz = []

for i,e in enumerate(selfembeds):
  embeddingz.append(e(categorical[:,i]))
embeddingz

[tensor([[-0.2673,  1.4414,  0.6270],
         [-0.2673,  1.4414,  0.6270],
         [-0.2673,  1.4414,  0.6270],
         ...,
         [-0.2673,  1.4414,  0.6270],
         [-0.2673,  1.4414,  0.6270],
         [-0.2673,  1.4414,  0.6270]], grad_fn=<EmbeddingBackward>),
 tensor([[0.8395],
         [0.8395],
         [0.8395],
         ...,
         [0.8395],
         [0.8395],
         [0.8395]], grad_fn=<EmbeddingBackward>),
 tensor([[ 0.1936, -0.7349],
         [ 0.1936, -0.7349],
         [-0.0283, -1.1927],
         ...,
         [ 0.1936, -0.7349],
         [ 0.1936, -0.7349],
         [ 0.1936, -0.7349]], grad_fn=<EmbeddingBackward>),
 tensor([[ 0.4245, -0.8125],
         [ 0.4245, -0.8125],
         [ 0.4245, -0.8125],
         ...,
         [ 0.4245, -0.8125],
         [ 0.4245, -0.8125],
         [ 0.4245, -0.8125]], grad_fn=<EmbeddingBackward>),
 tensor([[1.1731],
         [1.1731],
         [1.1731],
         ...,
         [1.1731],
         [1.1731],
         [1.1731]], g

In [33]:
z = torch.cat(embeddingz,1)  #Above we can see that there are different tensors for the 3 columns,so we merged them.
z, z.shape

(tensor([[-0.2673,  1.4414,  0.6270,  ..., -2.0226,  0.1430,  1.3109],
         [-0.2673,  1.4414,  0.6270,  ..., -2.0226,  0.1430,  1.3109],
         [-0.2673,  1.4414,  0.6270,  ..., -2.0226,  0.1430,  1.3109],
         ...,
         [-0.2673,  1.4414,  0.6270,  ..., -2.0226,  0.1430,  1.3109],
         [-0.2673,  1.4414,  0.6270,  ..., -2.0226,  0.1430,  1.3109],
         [-0.2673,  1.4414,  0.6270,  ..., -2.0226,  0.1430,  1.3109]],
        grad_fn=<CatBackward>), torch.Size([1338, 122]))

In [0]:
# converting everything to cuda

# For train set

categorical = categorical.cuda()
continous = continous.cuda()
y = y.cuda()

# For test set

cat_test = categorical_test.cuda()
cont_test = continous_test.cuda()

In [0]:
# Splitting the data into train and validation set
size = int(len(categorical)*0.85)
cat_train,cont_train, y_train = categorical[:size],continous[:size], y[:size]
cat_val,cont_val, y_val = categorical[size:],continous[size:], y[size:]

## 4) Building the Model

In [0]:
class ANN(nn.Module):
  def __init__(self,emb_size,n_cont,out_sz,layers,p=0.5):

    super().__init__()

    self.embeds = nn.ModuleList([nn.Embedding(n1,n2) for n1,n2 in emb_size])
    self.emb_drop = nn.Dropout(p=0.5)
    self.bn_cont = nn.BatchNorm1d(n_cont)

    layerlist = []
    n_emb = sum((j for i,j in emb_size))
    n_in = n_emb + n_cont

    tracker = 0
    for i in layers:
      layerlist.append(nn.Linear(n_in,i)) 
      layerlist.append(nn.ReLU(inplace=True))

      if tracker < 3:        
        layerlist.append(nn.BatchNorm1d(i))

      layerlist.append(nn.Dropout(p))
      n_in = i
      tracker +=1
    layerlist.append(nn.Linear(layers[-1],out_sz))
        
    self.layers = nn.Sequential(*layerlist)

  def forward(self,x_cat,x_cont):

    embeddings = []
    for i,e in enumerate(self.embeds):
        embeddings.append(e(x_cat[:,i]))
    x = torch.cat(embeddings, 1)
    x = self.emb_drop(x)
    
    x_cont = self.bn_cont(x_cont)
    x = torch.cat([x, x_cont], 1)
    x = self.layers(x)
    return x

In [38]:
# Initialising the model
model = ANN(emb_size,continous.shape[1],1,[300,300,200,100],p = 0.4)
gpu_model = model.cuda()
gpu_model

ANN(
  (embeds): ModuleList(
    (0): Embedding(5, 3)
    (1): Embedding(2, 1)
    (2): Embedding(4, 2)
    (3): Embedding(4, 2)
    (4): Embedding(2, 1)
    (5): Embedding(5, 3)
    (6): Embedding(3, 2)
    (7): Embedding(25, 13)
    (8): Embedding(9, 5)
    (9): Embedding(8, 4)
    (10): Embedding(5, 3)
    (11): Embedding(8, 4)
    (12): Embedding(6, 3)
    (13): Embedding(8, 4)
    (14): Embedding(14, 7)
    (15): Embedding(16, 8)
    (16): Embedding(4, 2)
    (17): Embedding(4, 2)
    (18): Embedding(4, 2)
    (19): Embedding(5, 3)
    (20): Embedding(4, 2)
    (21): Embedding(4, 2)
    (22): Embedding(4, 2)
    (23): Embedding(6, 3)
    (24): Embedding(6, 3)
    (25): Embedding(4, 2)
    (26): Embedding(5, 3)
    (27): Embedding(2, 1)
    (28): Embedding(5, 3)
    (29): Embedding(4, 2)
    (30): Embedding(7, 4)
    (31): Embedding(6, 3)
    (32): Embedding(3, 2)
    (33): Embedding(5, 3)
    (34): Embedding(5, 3)
    (35): Embedding(3, 2)
    (36): Embedding(9, 5)
    (37): Embed

## 5) Defining the loss function and Optimizer

In [0]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

## 6) Model Training

In [93]:
import time
start_time = time.time()

epochs = 5001
losses = []
gpu_model.train()

for i in range(epochs):
    i+=1
    y_pred = gpu_model(cat_train, cont_train)
    loss = torch.sqrt(criterion(y_pred, y)) # RMSE
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%200 == 0:
      gpu_model.eval()
      y_pred_val = gpu_model(cat_train, cont_train)
      losse = torch.sqrt(criterion(y_pred_val, y_val)) # RMSE
      print(f'epoch: {i}  loss: {losse.item()}')
      gpu_model.train()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'\nDuration: {(time.time() - start_time)/60} minutes') # print the time elapsed

epoch: 100  loss: 1.5740512609481812
epoch: 200  loss: 1.3338274955749512
epoch: 300  loss: 1.3836725950241089
epoch: 400  loss: 1.124122142791748
epoch: 500  loss: 0.9937739968299866
epoch: 600  loss: 0.9250100255012512
epoch: 700  loss: 0.8247978091239929
epoch: 800  loss: 0.754414975643158
epoch: 900  loss: 0.7881526947021484
epoch: 1000  loss: 0.7070537805557251
epoch: 1100  loss: 0.7138545513153076
epoch: 1200  loss: 0.652309000492096
epoch: 1300  loss: 0.6047720313072205
epoch: 1400  loss: 0.630083441734314
epoch: 1500  loss: 0.5605660676956177
epoch: 1600  loss: 0.5203043222427368
epoch: 1700  loss: 0.47192758321762085
epoch: 1800  loss: 0.4313240051269531
epoch: 1900  loss: 0.40979892015457153
epoch: 2000  loss: 0.37026849389076233
epoch: 2100  loss: 0.3198307454586029
epoch: 2200  loss: 0.274946391582489
epoch: 2300  loss: 0.2532244026660919
epoch: 2400  loss: 0.22417767345905304
epoch: 2500  loss: 0.18294382095336914
epoch: 2600  loss: 0.15927131474018097
epoch: 2700  loss: 0

In [105]:
#y_pred = y_pred.cpu()
#y_pred = y_pred.detach().numpy()
np.exp(y_pred)

array([[187681.17],
       [179910.19],
       [251648.64],
       ...,
       [261574.38],
       [137268.97],
       [138763.  ]], dtype=float32)

In [106]:
y = y.cpu()
y = y.detach().numpy()
np.exp(y)

array([[208499.94],
       [181499.95],
       [223499.92],
       ...,
       [266500.06],
       [142124.95],
       [147500.03]], dtype=float32)